# Draw Attention maps(가감된 어텐션)

In [1]:
import argparse
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pickle
import glob

from vit_rollout import VITAttentionRollout
from vit_grad_rollout import VITAttentionGradRollout
import vit_explain as vex

### Path Setting

In [6]:
PATH_model = "../../model/model.pt"

PATH_sample_data_TP_list = glob.glob("../../dataset/data_preprocessed/Vulner/21-12-01-11-41-59_end_extract_drive8/*") # 285~325
PATH_sample_data_TN_list = glob.glob("../../dataset/data_preprocessed/None-crash/21-12-01-11-07-44_end_extract_drive3/*") # 300~350
PATH_sample_data_FP_list = glob.glob("../../dataset/data_preprocessed/None-crash/21-12-29-11-04-19_end_extract_drive14/*") # 95~115
PATH_sample_data_FN_list = glob.glob("../../dataset/data_preprocessed/Vulner/21-12-14-15-04-44_end_extract_drive2/*") # 290~305 -> start with 292

PATH_sample_data_TN_list = PATH_sample_data_TN_list[297:347+1]
PATH_sample_data_FP_list = PATH_sample_data_FP_list[92:112+1]

### Load Model

In [ ]:
# using CPU
model = torch.load(PATH_model)
model.eval()
model.cpu()

### Draw

In [8]:
for data in PATH_sample_data_TP_list:
    vex.draw_img_sample_custom(model=model, PATH_data=data)

for data in PATH_sample_data_TN_list:
    vex.draw_img_sample_custom(model=model, PATH_data=data)

for data in PATH_sample_data_FP_list:
    vex.draw_img_sample_custom(model=model, PATH_data=data)

for data in PATH_sample_data_FN_list:
    vex.draw_img_sample_custom(model=model, PATH_data=data)